# Capstone Two #

## Predicting Test Performance for Specialised High School Admissions ##

Performance on the Specialized High School Admissions Test (SHSAT) determines eligibility to one of the eight specialized high schools in New York City. It is administered by the New York City Department of Education (DOE) to about a third of the city’s 8th graders, with 5,000 receiving admissions offers. Of major concern is the racial & ethnic breakdown of admitted students, showing significant underrepresentation from black and latinx students. 

The goal of this analysis is to elicit which factors predict performance on the SHSAT. These factors will serve as beacons to direct or draw services, whether education-based or otherwise, towards improving the percentage of black and latinx students admitted to the specialized high schools. 

This approach aims to quantify which variables lead to admissions offers beyond prior proxies: English Language Learners, Students with Disabilities, Students on Free/Reduced Lunch, and Students with Temporary Housing.

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas_profiling
import urllib.request
%matplotlib inline

### Data Collection & Loading Data Sets ###

Two main data sets will be used:

*2016 School Explorer*  
This dataset consists of 1272 schools in New York city, and 161 variables, provided via kaggle. Primarily, it’s school descriptors, e.g. grades, race & ethnicity student percentages, high/low performing percentages of students. Data is available as a single csv file.

*2017-2018 SHSAT Admissions Test Offers By Sending School*  
This dataset consists of the 2017 SHSAT results, published by NYC in 2018. All test takers are north of 28,000, grade 8 students, Test takers and offers received are grouped by school. Data is available as a single csv file from the NYC Open Data portal.

In [4]:
#downloading the 2016 School Explorer data set to raw folder
url = 'https://storage.googleapis.com/kagglesdsdata/datasets%2F33225%2F44131%2F2016%20School%20Explorer.csv?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595273466&Signature=fk6%2BI64ZSKXeenOP24Hsst9gNp2z3gFDGnz1rSXzcrnS874EFfR1VjUPVu0mCoN0bwXxJ7udjKpGlD51QLqiolRTpt9t%2F6ko672nNzd2KU0zJd4xRN8yW4Ouk1XxbCCTN2u6In241T1%2BY1RMpSp5rQgko83zQtwPClQPsl%2BWynlztsHV1aWF2K1J6MUy1SBaXyHvTSXBiMp1G2FvCoVjRVyjkXwV94Xgayi8Zgs3ISjyVUZn3yYzuyarl8NUwSnryWnfCE1debgt5z9AP5aTv7IbUA297hpYAhHZR0NjtKMwoadxypbWbBZ6cUTgI8KT4L4q8LdCgJ6SDJolkJERaQ%3D%3D'
file_explorer = '../data/raw/2016_School_Explorer.csv'
urllib.request.urlretrieve(url, file_explorer)

#downloading the 2017-2018 SHSAT data set to raw folder
url = 'https://data.cityofnewyork.us/resource/vsgi-eeb5.csv'
file_offers = '../data/raw/2017-2018_SHSAT_Admissions_Test_Offers_By_Sending_School.csv'
urllib.request.urlretrieve(url, file_offers)

In [5]:
#loading data sets
df_explorer = pd.read_csv(file_explorer)
df_offers = pd.read_csv(file_offers)

In [6]:
df_explorer.head(3)

,Adjusted Grade,New?,Other Location Code in LCGMS,School Name,SED Code,Location Code,District,Latitude,Longitude,Address (Full),...,Grade 8 Math - All Students Tested,Grade 8 Math 4s - All Students,Grade 8 Math 4s - American Indian or Alaska Native,Grade 8 Math 4s - Black or African American,Grade 8 Math 4s - Hispanic or Latino,Grade 8 Math 4s - Asian or Pacific Islander,Grade 8 Math 4s - White,Grade 8 Math 4s - Multiracial,Grade 8 Math 4s - Limited English Proficient,Grade 8 Math 4s - Economically Disadvantaged
0,NaN,NaN,NaN,P.S. 015 ROBERTO CLEMENTE,310100010015,01M015,1,40.721834,-73.978766,"333 E 4TH ST NEW YORK, NY 10009",...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,P.S. 019 ASHER LEVY,310100010019,01M019,1,40.729892,-73.984231,"185 1ST AVE NEW YORK, NY 10003",...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,NaN,P.S. 020 ANNA SILVER,310100010020,01M020,1,40.721274,-73.986315,"166 ESSEX ST NEW YORK, NY 10002",...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_offers.head()

,feeder_school_dbn,feeder_school_name,count_of_students_in_hs_admissions,count_of_testers,count_of_offers
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,58.0,6,0-5
1,01M140,P.S. 140 NATHAN STRAUS,67.0,6,0-5
2,01M184,P.S. 184M SHUANG WEN,88.0,67,23
3,01M188,P.S. 188 THE ISLAND SCHOOL,59.0,0-5,0-5
4,01M301,"TECHNOLOGY, ARTS, AND SCIENCES STUDIO",51.0,11,0-5


### Data Definitions ###

Generally exploring both datasets to understand what's in them.

In [28]:
df_explorer.columns

Index(['Adjusted Grade', 'New?', 'Other Location Code in LCGMS', 'School Name',
       'SED Code', 'Location Code', 'District', 'Latitude', 'Longitude',
       'Address (Full)',
       ...
       'Grade 8 Math - All Students Tested', 'Grade 8 Math 4s - All Students',
       'Grade 8 Math 4s - American Indian or Alaska Native',
       'Grade 8 Math 4s - Black or African American',
       'Grade 8 Math 4s - Hispanic or Latino',
       'Grade 8 Math 4s - Asian or Pacific Islander',
       'Grade 8 Math 4s - White', 'Grade 8 Math 4s - Multiracial',
       'Grade 8 Math 4s - Limited English Proficient',
       'Grade 8 Math 4s - Economically Disadvantaged'],
      dtype='object', length=161)

In [30]:
df_explorer.shape

(1272, 161)

In [44]:
df_explorer.describe().T

,count,mean,std,min,25%,50%,75%,max
SED Code,1272.0,3.286698e+11,1.222022e+10,3.075000e+11,3.208752e+11,3.315004e+11,3.424000e+11,3.531009e+11
District,1272.0,1.613522e+01,9.245270e+00,1.000000e+00,9.000000e+00,1.500000e+01,2.400000e+01,3.200000e+01
Latitude,1272.0,4.073454e+01,8.660234e-02,4.050780e+01,4.066950e+01,4.072299e+01,4.081563e+01,4.090346e+01
Longitude,1272.0,-7.391835e+01,8.057649e-02,-7.424402e+01,-7.395706e+01,-7.392057e+01,-7.387926e+01,-7.370892e+01
Zip,1272.0,1.081572e+04,5.295889e+02,1.000100e+04,1.045200e+04,1.120300e+04,1.123200e+04,1.169400e+04
...,...,...,...,...,...,...,...,...
Grade 8 Math 4s - Asian or Pacific Islander,1272.0,1.984277e+00,1.284133e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.460000e+02
Grade 8 Math 4s - White,1272.0,9.709119e-01,6.880223e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.260000e+02
Grade 8 Math 4s - Multiracial,1272.0,2.358491e-03,8.411582e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
Grade 8 Math 4s - Limited English Proficient,1272.0,1.595912e-01,1.321195e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.300000e+01


We can tell that there are 161 variables in the 2016 School Explorer data set with 1,272 rows of data consisting of physical descriptors and descriptive statistics information for all of the schools.

In [27]:
df_offers.columns

Index(['feeder_school_dbn', 'feeder_school_name',
       'count_of_students_in_hs_admissions', 'count_of_testers',
       'count_of_offers'],
      dtype='object')

In [31]:
df_offers.shape

(594, 5)

In [45]:
df_offers.describe()

,count_of_students_in_hs_admissions
count,593.000000
mean,127.008432
std,117.939653
min,9.000000
25%,63.000000
50%,88.000000
75%,132.000000
max,769.000000


The 2017-2018 SHSAT Admissions Test Offers By Sending School has 5 variables with 594 rows of data consisting of names and descriptive statistics for all of the schools. This makes sense because not all schools send students to the SHSAT.

**Initial Data Cleaning**

Let's determine the null values and then appropriately address what to do with them.

In [47]:
nas=pd.DataFrame(df_explorer.isnull().sum().sort_values(ascending=False)/len(df_explorer),columns = ['Percent'])
pos = nas['Percent'] > 0
nas[pos]

,Percent
Other Location Code in LCGMS,0.999214
Adjusted Grade,0.998428
New?,0.978774
School Income Estimate,0.311321
Student Achievement Rating,0.067610
Supportive Environment Rating,0.062893
Collaborative Teachers Rating,0.059748
Rigorous Instruction Rating,0.059748
Trust Rating,0.057390
Strong Family-Community Ties Rating,0.057390


We can see that the *explorer* data set has three columns almost entirely of null values. These can be filled with an appropriate value for the data type of those columns.

In [48]:
nas=pd.DataFrame(df_offers.isnull().sum().sort_values(ascending=False)/len(df_offers),columns = ['Percent'])
pos = nas['Percent'] > 0
nas[pos]

,Percent
count_of_students_in_hs_admissions,0.001684


The *offers* data set is pretty clean and doesn't need to have null values addressed.

In [63]:
#determine existing values of high-NaN columns identified above, for 2016 School Explorer dataframe
print(df_explorer['Other Location Code in LCGMS'].describe())
print(df_explorer['Adjusted Grade'].describe())
df_explorer['New?'].describe()

count          1
unique         1
top       75k372
freq           1
Name: Other Location Code in LCGMS, dtype: object
count     2
unique    1
top       x
freq      2
Name: Adjusted Grade, dtype: object


count     27
unique     1
top        x
freq      27
Name: New?, dtype: object

In [74]:
#replacing the null values with 'n' for no/ne
df_explorer['Other Location Code in LCGMS'].fillna('n', inplace=True)
df_explorer['Adjusted Grade'].fillna('n', inplace = True)
df_explorer['New?'].fillna('n', inplace=True)

The school income estimate column, which has more than 30% of its values equal to null, seems to not be a numeric data type.

In [137]:
df_explorer['School Income Estimate'].describe()

count             876
unique            876
top       $39,138.83 
freq                1
Name: School Income Estimate, dtype: object

In [128]:
df_explorer['School Income Estimate Numeric'] = df_explorer['School Income Estimate'].str.replace('$','').str.replace(',','').astype(float)

In [147]:
df_explorer['School Income Estimate Numeric'].fillna(0, inplace=True)
df_explorer['School Income Estimate Numeric'].isnull().sum()

0

Nice!